# Example for usage ARc Policy via a Strands Agent

Install requirements

In [ ]:
%pip install -r requirements.txt --upgrade

In [ ]:
# If needed, this will allow for the restart of the notebook

from IPython.display import display_html
def restart_kernel():
    display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)
restart_kernel()


## Setup for the agent

In [ ]:
from strands import Agent
from strands.models import BedrockModel
from strands.hooks import HookProvider, HookRegistry, MessageAddedEvent, BeforeModelCallEvent, BeforeToolCallEvent
from pydantic import BaseModel
import boto3
from botocore.config import Config as BotocoreConfig
from strands.telemetry import StrandsTelemetry
from findings_utils import extract_reasoning_findings
import logging
from strands_tools import retrieve
import os

# Configure the root strands logger
logging.getLogger("strands").setLevel(logging.DEBUG)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# Setup tracing - commented out for now as this adds a lot of trace output that really isn't interesting
StrandsTelemetry().setup_console_exporter()

# NOTE: To send the OTEL data to an ADOT collector, additional exporter needs to be used

## Provide values for the following cell

In [ ]:
# Supply the pre-installed polciy and guardrail IDs
ARC_POLICY_ARN = "<REPLACE ME>"
GUARDRAIL_ID = "<REPLACE ME>"
GUARDRAIL_VERSION = "<REPLACE ME>"
KNOWLEDGE_BASE_ID = "<REPLACE_ME>"
# NOTE: the default model for Strands is us.anthropic.claude-sonnet-4-20250514-v1:0
MODEL_ID = "us.amazon.nova-lite-v1:0"
# MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"


In [ ]:
# Setup the environment for the agent and tool
# Allow for the metadata to be retrieved on sources from the KB
os.environ['RETRIEVE_ENABLE_METADATA_DEFAULT'] = 'true'
# Allow for the retrieve tool to interact with the KB
os.environ['KNOWLEDGE_BASE_ID'] = KNOWLEDGE_BASE_ID

In [ ]:

# Define a notification hook to listen to events and then process the result and call
# Automated Reasoning attached via the Guardrail and report on the findings.  This
# can be used possibly re-write the output or add a flag on if the output is correct.
class NotifyOnlyGuardrailsHook(HookProvider):
    
    def __init__(self, guardrail_id: str, guardrail_version: str, arc_policy_arn: str):
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version
        self.arc_policy_arn = arc_policy_arn
        self.bedrock_client = boto3.client("bedrock-runtime")
        self.input = ''
        self.claim_valid = True
        self.findings = ''
        self.policy_definition = {}
        self.before_tool_event_flag = False
        self.before_model_event_flag = False

        if self.arc_policy_arn:
            try:
                bedrock_client = boto3.client('bedrock')
                response = bedrock_client.export_automated_reasoning_policy_version(policyArn=self.arc_policy_arn)
                self.policy_definition = response.get('policyDefinition', {})
            except Exception as e:
                print(f"Error getting policy definition: {str(e)}")
                raise

    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(BeforeModelCallEvent, self.before_model_event)
        registry.add_callback(BeforeToolCallEvent, self.before_tool_event)
        registry.add_callback(MessageAddedEvent, self.message_added)

    def message_added(self, event: MessageAddedEvent) -> None:
        if self.before_tool_event_flag:
            # Since a tool was called, just ignore this message addition
            self.before_tool_event_flag = False
            return
        
        # Get the content
        content = "".join(block.get("text", "") for block in event.message.get("content", []))

        # Determine the source
        if event.message.get("role") == "user":
            # Store the input for later usage and allow the loop to continue to process
            self.input = content
            return

        # Capture if this is the first time that findings will be created
        first_findings = (not self.findings)

        # Format a request to send to the guardrail
        content_to_validate = [
            {"text": {"text": self.input, "qualifiers": ["query"]}},
            {"text": {"text": content, "qualifiers": ["guard_content"]}}
        ]

        # Call the guardrail
        response = self.bedrock_client.apply_guardrail(
            guardrailIdentifier=self.guardrail_id,
            guardrailVersion=self.guardrail_version,
            source="OUTPUT",
            content=content_to_validate
        )

        # Determine if the output is correct
        self.findings = extract_reasoning_findings(response, self.policy_definition)
         
        assessments = response.get("assessments", [])
        if assessments and len(assessments):
            self.claim_valid = False

        # Add information to the output
        if self.findings and first_findings:
            new_output = content
            new_output = new_output + f"\n\nfindings: {self.findings}"
            new_output = new_output + f"\n\nclaim_valid: {self.claim_valid}"
            event.message["content"][0]["text"] = new_output
        
    def before_model_event(self, event: BeforeModelCallEvent) -> None:
        self.before_model_event_flag = True

    def before_tool_event(self, event: BeforeToolCallEvent) -> None:
        self.before_tool_event_flag = True

# Create structured output
class StructuredOutputModel(BaseModel):
    claim_valid: bool
    content: str
    findings: str

# Provide the config for botocore
boto_config = BotocoreConfig(
    retries={"max_attempts": 3, "mode": "standard"},
    connect_timeout=5,
    read_timeout=60
)

# Create a Bedrock model with guardrail configuration
bedrock_model = BedrockModel(
    boto_client_config=boto_config,
    model_id=MODEL_ID,
    # NOTE: An alternative option is to supply the guardrail here.  If going that route, the ARc findings aren't present.
    # To ensure that the findings are present and can be used to re-write the output, rely on a hook
)

# Create agent with the guardrail-protected model
auto_policy_agent = Agent(
    system_prompt="You are an assistant that determines if the users auto insurance claim is valid based on the provided information and details within the policy contract. In cases where a clear outcome is not present ask the user to check with their insurance agent. Take your time to think though the answer and evalute carefully.",
    model=bedrock_model,
    hooks=[NotifyOnlyGuardrailsHook(GUARDRAIL_ID, GUARDRAIL_VERSION, ARC_POLICY_ARN)],
    tools=[retrieve]
)



In [ ]:
# Ask the agent a question
auto_policy_agent("I was delivering goods for my company using my personal car. I got into an accident, this was over 3 months ago. Does my policy cover this?")

In [ ]:
# Get the structured output.  Without providing a prompt, the agent will act on conversation history to get the response
response = auto_policy_agent.structured_output(StructuredOutputModel)

# Extract the findings from the response
print(f"response.claim_valid = {response.claim_valid}")
print(f"repsonse.content = {response.content}")
print(f"repsonse.findings = {response.findings}")